# Attempt to use incomplete LLTF data to calibrate PISCES

In [1]:
import numpy as np
import glob
from astropy.io import fits
%pylab inline --no-import-all
plt.rc('font', family='serif', serif='Times',size=20)
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=20)
plt.rc('xtick.major', size=10)
plt.rc('ytick.major', size=10)
plt.rc('ytick', labelsize=20)
plt.rc('axes', labelsize=20)
plt.rc('figure',titlesize=20)
plt.rc('image',origin='lower',interpolation='nearest')

import sys
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
os.chdir('/Users/mrizzo/IFS/crispy/crispy/PISCES_LLTF/')

import logging as log
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.tools.image import Image
from PISCESparams import Params
par = Params()
par.hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                    2 / Interlacing                                    
PHILENS =    26.565051177077

## Collect LLTF data

In [9]:
import re
LLTF_folder = '/Users/mrizzo/IFS/PISCES/data_IFS_170626/'
LLTF_list = np.sort(glob.glob(LLTF_folder + 'Calibra_LLTF_170623_???nm_*'))
log.info(LLTF_list)
wavels = np.arange(600.,725.,10.)

crispy - INFO - [ '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_600nm_170623_144454_229546.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_600nm_170623_150738_918536.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_600nm_170623_154310_045190.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_610nm_170623_144607_899479.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_610nm_170623_150852_364952.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_610nm_170623_154422_973572.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_620nm_170623_144720_901586.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_620nm_170623_151005_846826.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_620nm_170623_154536_290824.fits'
 '/Users/mrizzo/IFS/PISCES/data_IFS_170626/Calibra_LLTF_170623_630nm_170623_144833_639517.fits'
 '/Users/mrizzo/IFS/PIS

In [11]:
dark = fits.getdata(LLTF_folder+'Dark_LLTF_170623_60s_170623_150506_052528.fits')
for i,wav in enumerate(wavels):
    j = i*3
    dat = fits.getdata(LLTF_list[j])
    dat+= fits.getdata(LLTF_list[j+1])
    dat+= fits.getdata(LLTF_list[j+2])
    dat /= 3.0
    dat -= dark
    out = fits.HDUList(fits.PrimaryHDU(dat))
    out.writeto(LLTF_folder+'spots_%s.fits' % (str(wav)),overwrite=True)

In [3]:
from crispy.tools.wavecal import buildcalibrations
par.gaussian_hires=False # since par.gaussian_hires is False, it will attempt to make high-resolution PSFLets
par.lamlist = np.arange(650.,725.,10)
par.filelist = [par.wavecalDir+'spots_'+str(wav)+'.fits' for wav in par.lamlist]
buildcalibrations(par,
                    inspect=True,
                    genwavelengthsol=True, # Compute wavelength at the center of all pixels
                    makehiresPSFlets=True, # this requires very high SNR on the monochromatic frames
                    makePolychrome=True,   # This is needed to use least squares extraction
                    makePSFWidths=True,    # Fit PSF widths from high-res PSFLet models
                    upsample=3,            # upsampling factor of the high-resolution PSFLets
                    nsubarr=4,             # the detector is divided into nsubarr^2 regions for PSFLet averaging
                    apodize=False,          # to match PSFlet spot locations, only use the inner circular part of the 
                                           #detector, hence discarding the corners of the detector where lenslets are 
                                           #distorted
                  ) 

crispy - INFO - Building calibration files, placing results in ..//ReferenceFiles/LLTFwavecal_attempt/
crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/LLTFwavecal_attempt/spots_650.0.fits
crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/LLTFwavecal_attempt/spots_650.0.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ..//ReferenceFiles/LLTFwavecal_attempt/spots_650.0.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ..//ReferenceFiles/LLTFwavecal_attempt/spots_650.0.fits
crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/LLTFwavecal_attempt/spots_660.0.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ..//ReferenceFiles/LLTFwavecal_atte